IMPORTS:

In [1]:
import requests
import io
import pandas as pd
import regex as re
import numpy as np
import json
from statistics import multimode

from bs4 import BeautifulSoup
from pdfminer.high_level import extract_text
#import PyPDF2 as pdf2

import pycountry

from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import snowballstemmer
from keybert import KeyBERT



In [ ]:
%store -r date_df_ok

VARIABLES:

In [6]:
import regex as re

In [2]:
[cntry.name for cntry in list(pycountry.countries)] + ["USA"]

['Aruba',
 'Afghanistan',
 'Angola',
 'Anguilla',
 'Åland Islands',
 'Albania',
 'Andorra',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'American Samoa',
 'Antarctica',
 'French Southern Territories',
 'Antigua and Barbuda',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Burundi',
 'Belgium',
 'Benin',
 'Bonaire, Sint Eustatius and Saba',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'Bahrain',
 'Bahamas',
 'Bosnia and Herzegovina',
 'Saint Barthélemy',
 'Belarus',
 'Belize',
 'Bermuda',
 'Bolivia, Plurinational State of',
 'Brazil',
 'Barbados',
 'Brunei Darussalam',
 'Bhutan',
 'Bouvet Island',
 'Botswana',
 'Central African Republic',
 'Canada',
 'Cocos (Keeling) Islands',
 'Switzerland',
 'Chile',
 'China',
 "Côte d'Ivoire",
 'Cameroon',
 'Congo, The Democratic Republic of the',
 'Congo',
 'Cook Islands',
 'Colombia',
 'Comoros',
 'Cabo Verde',
 'Costa Rica',
 'Cuba',
 'Curaçao',
 'Christmas Island',
 'Cayman Islands',
 'Cyprus',
 'Czechia',
 'Germany',
 'Djibouti',
 'Dominica'

In [7]:
# World Univ. JSON Related Variables:

# Uni Email Codes-Country dict:
# Uni Names-Country dict:

def uni_dict_creator():
    with open('world_universities_and_domains.json', encoding="utf8") as fp:
        uni_dict = json.load(fp)

    uni_df = pd.DataFrame(uni_dict)

    uni_code_country_dict = {}
    uni_name_country_dict = {}

    for index, row in uni_df.iterrows():
        uni_code_country_dict.update({dom : row.country for dom in row.domains})
        uni_name_country_dict.update({row["name"] : row.country})

    del uni_name_country_dict["University of Technology"]

    return uni_code_country_dict, uni_name_country_dict

#for test reasons:
uni_code_country_dict, uni_name_country_dict = uni_dict_creator()

# pycountry Related Variables:
# Country Name list:
cntry_name_list = [cntry.name for cntry in list(pycountry.countries)] + ["USA"]
# Country Email Code (alpha 2)-Country Dict:
cntry_code_dict = {cntry.alpha_2.lower() : cntry.name for cntry in list(pycountry.countries)}


# REGEX PATTERNS:
# Email Regex Pattern:
email_reg = re.compile("[a-zA-Z0-9._-]+@([a-zA-Z0-9_-]+\.((?:[a-zA-Z0-9_-]+\.)*([a-zA-Z0-9_-]+)))")
# Uni Names Regex Pattern:
uni_name_regex = re.compile("(?=("+'|'.join(list(uni_name_country_dict.keys()))+r"))")
# Country Name Regex Pattern:
country_name_regex = re.compile("(?=("+'|'.join(cntry_name_list)+r"))")


In [13]:
# 1- E-Mail(Uni Domain) - DONE! :

def email_converter(pdf_read):
    
    email_caught = email_reg.search(pdf_read)

    try:
        return uni_code_country_dict[email_caught.group(1).lower()]
    except AttributeError:
        return np.nan
    except KeyError:
        try:
            return uni_code_country_dict[email_caught.group(2).lower()]
        except KeyError:
            try:
                return cntry_code_dict[email_caught.group(3)]
            except KeyError:
                cntry_pttrn =  email_caught.group(3)[:3]

                if cntry_pttrn in ["com","net"]:
                    return np.nan

                elif cntry_pttrn in ["edu","gov"]:
                    return "United States"

                if len(cntry_pttrn)==3:
                    cntry_pttrn = cntry_pttrn[:-1]

                try:
                    return cntry_code_dict[cntry_pttrn]
                except:
                    return np.nan


In [14]:
email_converter(email_testo)

'Germany'

In [31]:
# 2- Country Match - DONE:
def country_converter(pdf_read):
    try:
        cntry_match = country_name_regex.search(pdf_read)[1]
        return  cntry_match if cntry_match != "USA" else "United States"
    except TypeError:
        try:
            return country_name_regex.search(pdf_read.title())[1]
        except:
            return np.nan

In [48]:
# 3- University Match - DONE! :
def univ_converter(pdf_read):   
    try:
        uni_found = uni_name_regex.search(pdf_read)[1]
        return uni_name_country_dict[uni_found]
    except:
        return np.nan

In [21]:
def get_affiliation(pdf_read):
    email_affl = email_converter(pdf_read)
    uni_affl = univ_converter(pdf_read)
    cntry_affl = country_converter(pdf_read)

    affl_vote = multimode([email_affl,uni_affl,cntry_affl])

    if len(affl_vote) == 1:
        return affl_vote[0]
    elif not np.nan(email_affl):
        return email_affl
    elif not np.nan(uni_affl):
        return uni_affl
    else:
        return cntry_affl

All three Affiliation capture funcitons are ready! Required variables are:

* "World Unis" json & pycountry
* E-mail fn:
    - email_reg_modified
    - uni_code_country_dict
    - country_code_dict
* Country fn:
    - cntry_name_list
* Uni fn:
    - uni_name_list
    - uni_name_country_dict

DATE REGEXES:

In [51]:
# REGEX PATTERNS - RECEIVED & SUBMITTED

# 1 - 10 MAY 1960:
rec_1 = re.compile("(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)(\d{1,2}\s*(Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober)?|Nov(ember|.)?|Dec(ember|.)?)\s*(19|20)\d{2})", re.IGNORECASE)

# 2 - MAY 10, 1960
rec_2 = re.compile("(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)((Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober|.)?|Nov(ember|.)?|Dec(ember|.)?)\s?\d{1,2}\,?\s?(19|20)\d{2})", re.IGNORECASE)

# 3 - MAY 1960
rec_3 = re.compile("(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)((Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober|.)?|Nov(ember|.)?|Dec(ember|.)?)\s*(19|20)\d{2})", re.IGNORECASE)

# 4 - 10/05/1960 OR 10.05.1960 OR 10/05/1960
rec_4 = re.compile("(?:(?:received|submitted)\s?(?:on|date)?\s?\:?\s?)([1-3][0-9](\.|\-|\/|\:)?[0-1][0-9](\.|\-|\/\:)?(19|20)\d{2})", re.IGNORECASE)


In [62]:
def rec_date_capture(pdf_read):

    rec_patterns = [rec_1, rec_2, rec_3, rec_4]

    for pattern in rec_patterns:
        try:
            date = pattern.search(pdf_read).group(1)
        except AttributeError:
            pass
        else:
            return date



In [69]:
# REGEX PATTERNS - ACCEPTED

# 1 - 10 MAY 1960:
acc_1 = re.compile("(?:(?:accepted)\s?(?:on|date|for publication on)?\s?\:?\s?)(\d{1,2}\s*(Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober|.)?|Nov(ember|.)?|Dec(ember|.)?)\s*(19|20)\d{2})", re.IGNORECASE)

# 2 - MAY 10, 1960
acc_2 = re.compile("(?:(?:accepted)\s?(?:on|date|for publication on)?\s?\:?\s?)((Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober|.)?|Nov(ember|.)?|Dec(ember|.)?)\s?\d{1,2}\,?\s?(19|20)\d{2})", re.IGNORECASE)

# 3 - MAY 1960
acc_3 = re.compile("(?:(?:accepted)\s?(?:on|date|for publication on)?\s?\:?\s?)((Jan(uary|.)?|Feb(ruary|.)?|Mar(ch|.)?|Apr(il|.)?|May|Jun(e|.)?|Jul(y|.)?|Aug(ust|.)?|Sep(tember|.)?|Oct(ober|.)?|Nov(ember|.)?|Dec(ember|.)?)\s*(19|20)\d{2})", re.IGNORECASE)

# 4 - 10/05/1960 OR 10.05.1960 OR 10/05/1960
acc_4 = re.compile("(?:(?:accepted)\s?(?:on|date|for publication on)?\s?\:?\s?)([1-3][0-9](\.|\-|\/|\:)?[0-1][0-9](\.|\-|\/\:)?(19|20)\d{2})", re.IGNORECASE)


In [70]:

def acc_date_capture(pdf_text):
    
    acc_patterns = [acc_1, acc_2, acc_3, acc_4]
    
    for pattern in acc_patterns:
        try:
            date = pattern.search(pdf_text).group(1)
        except AttributeError:
            pass
        else:
            return date

PDF WORKS:

In [ ]:
# Simple Preprocessing:
def prep_simple(pdf_read):
    pdf_read = pdf_read.replace("\n"," ")
    pdf_read = re.sub(' +', ' ', pdf_read)
    return pdf_read


In [ ]:
def remove_references(pdf_read):
    # Removing "References":
    pdf_modified = "".join(pdf_read.split("references")[:-1])
    
    if len(pdf_modified) > 0:
        return pdf_modified 
    else:
        pdf_modified = "".join(pdf_read.split("acknowledgements")[:-1])
        if len(pdf_modified) > 0:
            return pdf_modified 
        else:
            return pdf_read



In [ ]:
# Main Preprocessing:

# pdf_read -> pdf_modified

def prep_main(pdf_modified):
    # Lowercase all:
    pdf_modified = pdf_modified.lower()
    # Removing "References":
    pdf_modified = remove_references(pdf_modified)
    # Remove URL:
    pdf_modified = re.sub(r'http\S+', '', pdf_modified)
    # Remove emails:
    pdf_modified = re.sub(r"\S*@\S*\s?", "", pdf_modified)
    # Remove everything in brackets & paranthesis:
    pdf_modified = re.sub("[\(\[].*?[\)\]]", "", pdf_modified)
    # Remove punctuation:
    pdf_modified = re.sub(r'[^\w\s]', '', pdf_modified)
    # Remove numbers
    pdf_modified = re.sub(r'[0-9]', '', pdf_modified)
    # Remove HTML tags:
    pdf_modified = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f-\xff]', '', pdf_modified)
    # Remove single letters:
    pdf_modified = re.sub(r"(?<=(^|\s))\D(\s|$)","", pdf_modified)
    # Remove multiple spaces:
    pdf_modified = re.sub(' +', ' ', pdf_modified)


In [80]:
len("".join("hasjkdhassasdgas".split("references")[:-1]))

0

TESTING AREA:

In [ ]:
# Get a random pdf_read:

random_dir_url = date_df_ok[date_df_ok.Rec_date.notna()].direct_url.sample(1).values[0]

HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36'}

pdf_req = requests.get(random_dir_url,headers=HEADERS)

print(pdf_req)
pdf_io = io.BytesIO(pdf_req.content)
pdf_read = extract_text(pdf_io,page_numbers=[])

print(random_dir_url)


In [ ]:
def get_keywords(pdf_read):
    kw_model = KeyBERT()
    keywords = kw_model.extract_keywords(pdf_read, keyphrase_ngram_range=(1, 1), stop_words="english", top_n=10)
    return keywords


In [87]:
class ogi():
    var1 = "vok" 
    
    def og_printer(self):
        print(ogi.var1)

In [88]:
og = ogi()

In [89]:
og.og_printer()

vok


In [ ]:
    @retry((OgiError),tries=5, delay=75)
    @retry((FourZeroFourError),tries=5, delay=15)
    def get_pdf_bytes(self, pdf_url):

        try:
            pdf_cont = self.sess.get(pdf_url, verify=False, timeout=15)

            if pdf_cont.status_code == 404:
                self._change_base_url()
                raise FourZeroFourError("Threat Level: 404")

        except:
            self._change_base_url()
            raise FourZeroFourError(f"Can't connect to direct URL: {pdf_url}")

        else:
            try:
                pdf_bytes = io.BytesIO(pdf_cont.content)
                return pdf_bytes
            except:
                self._change_base_url()
                raise FourZeroFourError("can't get bytes")

In [ ]:
    @retry((OgiError), tries=3, delay=5)
    def pdf_reader(self,pdf_bytes):
        try:
            pdf_read = extract_text(pdf_bytes)
            return pdf_read
        except:
            self._change_base_url()
            raise OgiError("can't read pdf")

In [ ]:
    def get_all(self, identifier):

        try:
            pdf_bytes = self.get_pdf_bytes(identifier)       
        except FourZeroFourError:
            return "direct_url_error"

        try:
            pdf_read = self.pdf_reader(pdf_bytes)
            
        except OgiError:
            return "cant_read_pdf"

        else:
            pdf_read = prep_simple(pdf_read)

            dates = self.get_dates(pdf_read)
            affiliations = self.get_affiliations(pdf_read)

            pdf_read = self.prep_main(pdf_read)
            keywords = self.get_keywords(pdf_read)

            return dates, affiliations, keywords



In [ ]:
    def rec_date_capture(self,pdf_read):
        rec_patterns = [self.rec_1, self.rec_2, self.rec_3, self.rec_4]
        for pattern in rec_patterns:
            try:
                date = pattern.search(pdf_read).group(1)
            except AttributeError:
                pass
            else:
                return date

    def acc_date_capture(self,pdf_text):
        acc_patterns = [self.acc_1, self.acc_2, self.acc_3, self.acc_4]
        for pattern in acc_patterns:
            try:
                date = pattern.search(pdf_text).group(1)
            except AttributeError:
                pass
            else:
                return date


In [ ]:
def get_dates(self,pdf_read):
    rec_date = self.rec_date_capture(pdf_read)
    acc_date = self.acc_date_capture(pdf_read)
    return [rec_date, acc_date] if (rec_date != None) | (acc_date != None) else ["no_date_found"]

In [1]:
from scihub_wartortle import SciHub_watergun

In [2]:
sh_w = SciHub_watergun()

In [8]:
random_dir_url = date_df_ok[date_df_ok.Rec_date.notna()].direct_url.sample(1).values[0]
print(random_dir_url)

https://www.nature.com/articles/eye201520.pdf


In [4]:
sh_w.get_all(random_dir_url)

(['19 July 2014', '19 January 2015'],
 'United States',
 [('vitrectomy', 0.4571),
  ('vitreoretinopathy', 0.4481),
  ('ophthalmology', 0.4463),
  ('retinopathy', 0.4418),
  ('melanomas', 0.4251),
  ('melanoma', 0.3913),
  ('ocular', 0.3882),
  ('uveal', 0.3726),
  ('cataract', 0.3696),
  ('intraocular', 0.3585)])

In [5]:
%store -r date_df_ok

In [12]:
testo_dict = [{"harf": "a", "rank": 1},{"harf": "b", "rank": 2},{"harf": "c", "rank": 3},{"harf": "d", "rank": 4}]

In [7]:
def testofn(rank):
    iki = rank % 2
    uc = rank % 3
    return iki, uc

In [13]:
for harf in testo_dict:
    ikires, ucres = testofn(harf["rank"])
    harf["ikires"] = ikires
    harf["uc"] = ucres

In [14]:
testo_dict

[{'harf': 'a', 'rank': 1, 'ikires': 1, 'uc': 1},
 {'harf': 'b', 'rank': 2, 'ikires': 0, 'uc': 2},
 {'harf': 'c', 'rank': 3, 'ikires': 1, 'uc': 0},
 {'harf': 'd', 'rank': 4, 'ikires': 0, 'uc': 1}]